In [1]:
import torch, torchdata, torchtext
import torch.nn as nn
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__, torchdata.__version__, torchtext.__version__

('2.2.2+cu121', '0.7.1', '0.17.2+cpu')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
SEED = 1234 #change three times
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 1. load dataset

In [5]:
df = pd.read_json('data/train-qar.jsonl', lines=True, nrows=300000)
df = df[['category', 'questionText']]
df.head(5)

,category,questionText
0,Toys_and_Games,"Many have stated similar to the following: ""Pa..."
1,Health_and_Personal_Care,Will these work with the Phillips sonicare han...
2,Cell_Phones_and_Accessories,What kind of sim card it use?
3,Home_and_Kitchen,does anyone know if this dinnerware set does n...
4,Musical_Instruments,I'm thinking of getting in to modular synthesi...


In [6]:
# Product category
df['category'].unique()

array(['Toys_and_Games', 'Health_and_Personal_Care',
       'Cell_Phones_and_Accessories', 'Home_and_Kitchen',
       'Musical_Instruments', 'Baby', 'Sports_and_Outdoors',
       'Patio_Lawn_and_Garden', 'Video_Games', 'Pet_Supplies',
       'Tools_and_Home_Improvement', 'Beauty', 'Electronics',
       'Grocery_and_Gourmet_Food', 'Automotive', 'Office_Products',
       'Clothing_Shoes_and_Jewelry'], dtype=object)

In [7]:
# convert the text to numeric class
class_mapping = {
    'Toys_and_Games': 0,
    'Health_and_Personal_Care': 1,
    'Cell_Phones_and_Accessories': 2,
    'Home_and_Kitchen': 3,
    'Musical_Instruments': 4,
    'Baby': 5,
    'Sports_and_Outdoors': 6,
    'Patio_Lawn_and_Garden': 7,
    'Video_Games': 8,
    'Pet_Supplies': 9,
    'Tools_and_Home_Improvement': 10,
    'Beauty': 11,
    'Electronics': 12,
    'Grocery_and_Gourmet_Food': 13,
    'Automotive': 14,
    'Office_Products': 15,
    'Clothing_Shoes_and_Jewelry': 16
}

# Map class names to numerical labels
df['category'] = df['category'].map(class_mapping)

In [8]:
# sample 100 data in each classes
df_sample = df.groupby('category', group_keys=False).apply(lambda x: x.sample(500))

In [9]:
# convert to lower case
df_sample['questionText']  =  df_sample['questionText'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [10]:
def data_cleaning(data):
    regex_s = re.sub("\\(.+?\\)|[\r\n|\n\r]|!", "", data)
    fin = " ".join(regex_s.split())
    return fin

In [11]:
df_sample['questionText'] = df_sample['questionText'].apply(data_cleaning)

In [12]:
train_df, val_df = train_test_split(df_sample, test_size=0.1,stratify=df_sample['category'], random_state=SEED)

In [13]:
train_df, test_df = train_test_split(train_df, test_size=0.1, stratify=train_df['category'],random_state=SEED)

In [14]:
train_df['category'].value_counts()

category
9     405
0     405
1     405
15    405
6     405
12    405
10    405
4     405
13    405
8     405
14    405
5     405
11    405
16    405
2     405
3     405
7     405
Name: count, dtype: int64

In [15]:
train_df

,category,questionText
97766,9,could this cage be considered cat proof? if no...
94610,8,will it work with kindle fire hdx
264351,3,is the powerplug australian fit?
228098,2,how is the connection when running outside wit...
273666,16,can you put a water bladder in this bag?
...,...,...
25733,10,is this also suitable for paint mixing?
55423,5,is this wrap easy to breastfeed in? i was look...
17908,10,does it lock in the off position so you can us...
29610,2,what does oem mean? will this work with the sa...


## 2.Preprocessing

In [16]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

## 3. Data loader

In [17]:
bert_model_name = 'bert-base-uncased'
max_length = 128
batch_size = 16

In [18]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_df['questionText'].tolist(), train_df['category'].tolist(), tokenizer, max_length)
val_dataset = TextClassificationDataset(val_df['questionText'].tolist(), val_df['category'].tolist(), tokenizer, max_length)
test_dataset = TextClassificationDataset(test_df['questionText'].tolist(), test_df['category'].tolist(), tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

## 4. Model (BERT)

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        for param in self.bert.parameters():
            param.requires_grad = False
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

## 5. Train

In [20]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    epoch_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        
        correct_predictions += torch.sum(preds == labels).item()
        total_predictions += len(labels)
        
        loss = nn.CrossEntropyLoss()(outputs, labels)
        epoch_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    accuracy = correct_predictions / total_predictions
    epoch_loss /= len(data_loader)  # Average loss per batch
    return epoch_loss,accuracy

In [21]:
def evaluate(model, data_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    epoch_loss = 0
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            epoch_loss += loss.item()
            
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_predictions += len(labels)
    
    accuracy = correct_predictions / total_predictions
    epoch_loss /= len(data_loader)  # Average loss per batch
    return  epoch_loss,accuracy

### Actual training

In [22]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
#experiment tracking
import mlflow
from mlflow.models import infer_signature
import os

# This the dockerized method.
# We build two docker containers, one for python/jupyter and another for mlflow.
# The url `mlflow` is resolved into another container within the same composer.
mlflow.set_tracking_uri("http://mlflow:5000")
# In the dockerized way, the user who runs this code will be `root`.
# The MLflow will also log the run user_id as `root`.
# To change that, we need to set this environ["LOGNAME"] to your name.
os.environ["LOGNAME"] = "noppawee"
#mlflow.create_experiment(name="noppawee-ML-project")  #create if you haven't create
mlflow.set_experiment(experiment_name="BERT500")


/usr/local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


<Experiment: artifact_location='mlflow-artifacts:/252120056759092590', creation_time=1714100306850, experiment_id='252120056759092590', last_update_time=1714100306850, lifecycle_stage='active', name='BERT500', tags={}>

In [24]:
bert_model_name = 'bert-base-uncased'
num_classes = 17
num_epochs = 5
learning_rate = 1e-3

In [25]:
model = BERTClassifier(bert_model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
import time
import torch.optim as optim



params={"model":"BERT", "num_epochs":num_epochs,"lr":learning_rate}
mlflow.start_run(run_name=f"BERT500-{params['num_epochs']}-epochs-lr-{params['lr']}-17-classes")
mlflow.log_params(params)



train_losses, train_accs, val_losses, val_accs = [],[],[],[]
best_valid_loss = float('inf')

for epoch in range(num_epochs):
    start_time = time.time()
                
    train_loss, train_acc = train(model, train_dataloader, optimizer, scheduler, device)
    valid_loss, valid_acc = evaluate(model, val_dataloader, device)
    #for plotting
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(valid_loss)
    val_accs.append(valid_acc)
                
    end_time = time.time()
                
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    mlflow.log_metric(key="train_loss", value=train_loss, step=epoch)
    mlflow.log_metric(key="train_acc", value=train_acc, step=epoch)
    mlflow.log_metric(key="val_loss", value=valid_loss, step=epoch)
    mlflow.log_metric(key="val_acc", value=valid_acc, step=epoch)
                
            
    #early stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        mlflow.pytorch.log_model(model, "model")
                
    print(f'Epoch: {epoch+1} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal.  Loss: {valid_loss:.3f} | Val Acc: {valid_acc*100:.2f}%')
mlflow.log_metric(key="min_val_loss", value=min(val_losses), step=epoch)    
mlflow.end_run()
        

Epoch: 1 | Time: 8m 19s
	Train Loss: 2.876 | Train Acc: 7.81%
	Val.  Loss: 2.793 | Val Acc: 7.41%


/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Epoch: 2 | Time: 8m 24s
	Train Loss: 2.784 | Train Acc: 10.99%
	Val.  Loss: 2.773 | Val Acc: 9.88%


KeyboardInterrupt: 

In [ ]:
# def predict_sentiment(text, model, tokenizer, device, max_length=128):
#     model.eval()
#     encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
#     input_ids = encoding['input_ids'].to(device)
#     attention_mask = encoding['attention_mask'].to(device)

#     with torch.no_grad():
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         _, preds = torch.max(outputs, dim=1)
#     return "positive" if preds.item() == 1 else "negative"